In [1]:
import os
os.chdir("../")
%pwd

'h:\\Personal Projects\\Delhi-Housing-Prices-Backend'

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [3]:
from houseProject.constants import *
from houseProject.utils.common import read_yaml, create_directories

In [4]:
class ConfigurationManager:
    
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [5]:
import os
import urllib.request as request
import zipfile
from houseProject import logger
from houseProject.utils.common import get_size

In [6]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [7]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-05-18 21:29:27,127: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-05-18 21:29:27,130: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-18 21:29:27,146: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-05-18 21:29:27,149: INFO: common: created directory at: artifacts]
[2025-05-18 21:29:27,151: INFO: common: created directory at: artifacts/data_ingestion]
[2025-05-18 21:29:28,186: INFO: 2280763589: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 112518
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "691ac908c0becb0d4f1567c901012a69e6cf15b579b6e38a13a3d75f1dc9e0ed"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: CB20:408C9:25CE89:61C794:682A03DF
Accept-Ranges: bytes
Date: Sun, 18